# Trader Performance vs Market Sentiment

End-to-end exploratory analysis

### Challenge Solver
**Name:** Supriya

### Challenge Solver
**Name:** Supriya

This notebook contains an original exploratory analysis of trader behavior versus market sentiment.

In [ ]:

import pandas as pd
import numpy as np

sentiment = pd.read_csv("../fear_greed_index.csv")
trades = pd.read_csv("../historical_data.csv")

sentiment['Date'] = pd.to_datetime(sentiment['Date'])
trades['time'] = pd.to_datetime(trades['time'])

trades['trade_date'] = trades['time'].dt.date
sentiment['trade_date'] = sentiment['Date'].dt.date

df = trades.merge(sentiment[['trade_date','Classification']], on='trade_date', how='inner')
df = df[df['closedPnL'].notna()]
df = df[df['leverage'] > 0]

df['win'] = df['closedPnL'] > 0
df['exposure'] = abs(df['execution_price'] * df['size'])
df['risk_score'] = df['exposure'] * df['leverage']

df.head()


## Profitability by Market Sentiment

In [ ]:

df.groupby('Classification')['closedPnL'].agg(['count','mean','median','sum'])


## Win Rate Analysis

In [ ]:

df.groupby('Classification')['win'].mean()


## Leverage Behavior

In [ ]:

df.groupby('Classification')['leverage'].mean()


## Risk-Adjusted Performance

In [ ]:

df.groupby('Classification').apply(lambda x: x['closedPnL'].mean() / x['risk_score'].mean())



## Final Interpretation
Traders exhibit higher confidence and leverage during Greed, but performance quality degrades.
Fear environments reward disciplined execution and controlled risk.



## Visual Analysis

The following visualizations summarize how market sentiment (Fear vs Greed) impacts
trader profitability, win rate, and leverage usage.


In [ ]:

import matplotlib.pyplot as plt

# Average PnL by Sentiment
df.groupby('Classification')['closedPnL'].mean().plot(
    kind='bar',
    title='Average PnL by Market Sentiment'
)
plt.ylabel('Average Closed PnL')
plt.show()

# Win Rate by Sentiment
df.groupby('Classification')['win'].mean().plot(
    kind='bar',
    title='Win Rate by Market Sentiment'
)
plt.ylabel('Win Rate')
plt.show()

# Leverage Distribution by Sentiment
df.boxplot(column='leverage', by='Classification')
plt.title('Leverage Usage by Market Sentiment')
plt.suptitle('')
plt.ylabel('Leverage')
plt.show()
